# Elementary data manipulation: Split-Apply-Combine with `pandas` and `numpy`

## 0. Standard data types and structures

### 0.1 Data types
+ boolean: `TRUE(1)` 또는 `FALSE(0)`의 값을 가지는 논리값으로, 정수로도 취급됨
+ integer: 정수 (unbounded)
+ float: 소수점을 가지는 일체의 유리수
+ string: 문자형

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # Jupyter notebook/IPython notebook에서 모든 행의 실행 결과를 보여주기 위한 설정
import numpy as np
import pandas as pd
import dfply as dp
import matplotlib.pyplot as pyplot

In [2]:
type(True)
type(1)
type(bool(1047)) # 정수형을 boolean으로 선언하면 1 이상의 모든 값은 True가 됨
type(2^30)
type(int(3)) # 명시적으로 정수형 선언
type(int(2^256)) # 명시적으로 정수형 선언: 오류
type(1.03)
type(4/7)
type(5.67e+04) #e식 표현법은 부동소수점 표현법이라 하며 이 수는 56700과 같음
type('My name is Lora')

bool

int

bool

int

int

int

float

float

float

str

## `pandas.Categorical`

In [3]:
from pandas import Categorical as cat

In [4]:
ftest = cat(['Green','Blue','Red'])
ftest
ftest2 = cat(['Green', 'Blue', 'Red'], ordered = True)
ftest2
ftest2_1 = cat(['Green', 'Blue', 'Red'], categories= ['Red', 'Green', 'Blue'])
ftest2_1
ftest3 = cat(range(1,6), categories = [3,1,5,4,2], ordered = True)
ftest3

[Green, Blue, Red]
Categories (3, object): [Blue, Green, Red]

[Green, Blue, Red]
Categories (3, object): [Blue < Green < Red]

[Green, Blue, Red]
Categories (3, object): [Red, Green, Blue]

[1, 2, 3, 4, 5]
Categories (5, int64): [3 < 1 < 5 < 4 < 2]

### Why do we use `Categorical`?
- 메모리 절약: 실제로 저장되는 것은 모든 `catogries`의 레이블에 대응하는 정수이기 때문입니다.
- 아래 코드 블록과 실행 결과를 참고하시면 확실하게 이해되시리라 생각합니다.
    - `ordered`와 `categories`가 별도로 지정된 경우, 차지하는 메모리 용량이 좀 더 늘어난 것을 알 수 있습니다.
- `sys.getsizeof` 함수는 `sys` 모듈을 불러들여서 실행할 수 있으며, 객체의 메모리 상 크기를 반환합니다 (byte 단위).

In [5]:
import sys, os
sys.getsizeof(np.repeat(['Green', 'Blue', 'Red'], [1000, 250, 400]))
sys.getsizeof(cat(np.repeat(['Green', 'Blue', 'Red'], [1000, 250, 400])))
sys.getsizeof(cat(np.repeat(['Green', 'Blue', 'Red'], [1000, 250, 400]), categories = ['Green', 'Red', 'Blue'], ordered = True))

33096

1937

1967

## 0.2 Data classes
### 0.2.1 `plain`
+ `list`: 자료의 뭉치
+ `dict`(dictionary): 이름과 항목이 대응되는 정보 뭉치

### 0.2.2 `numpy`
+ `array`: 1차원 숫자 뭉치
    + `matrix`: `array`에서 유도된 행렬형의 자료 저장형 (`matrix`에 딸린 attributes와 method를 사용할 수 있음)
+ `ndarray`: 다차원 숫자 뭉치

### 0.2.3 `pandas`
+ `DataFrame`

In [6]:
type([1, 2, 3])
type(['Alpha','Bravo','Charlie'])
mat = np.matrix('1 3 3;2 4 4;2 1 2')
mat
type(mat)
mat.base
mat.I # inverse 
mat.T # transpose
mat.getA() # matrix를 ndarray로
mat.getA1() # matrix를 row-wise로 이어붙인 array로
mat2 = np.matrix(np.random.randint(low = 1, high = 16, size = (4, 4))) # numpy.random: 난수 생성에 관한 submodule
mat2
mat2.getA()
list_arr = [mat, mat2] # matrices를 이어붙인 list는?
type(list_arr) # list!

list

list

matrix([[1, 3, 3],
        [2, 4, 4],
        [2, 1, 2]])

numpy.matrixlib.defmatrix.matrix

array([[1, 3, 3],
       [2, 4, 4],
       [2, 1, 2]])

...

array([[1, 3, 3],
       [2, 4, 4],
       [2, 1, 2]])

array([1, 3, 3, 2, 4, 4, 2, 1, 2])

matrix([[ 8,  8,  7, 10],
        [ 7,  3, 15,  5],
        [11,  9,  9, 11],
        [ 8,  3,  9, 10]])

array([[ 8,  8,  7, 10],
       [ 7,  3, 15,  5],
       [11,  9,  9, 11],
       [ 8,  3,  9, 10]])

list

In [7]:
## pandas.DataFrame 정의의 예
dat = pd.DataFrame({'field1': ['doc', 'dog', 'dor'],
                  'field2': range(0, 3),
                  'field3': cat(['alpha', 'beta', 'phi'], categories = ['phi', 'beta', 'alpha'])})
dat
dat.info() # 각 필드에 담긴 정보의 간략한 정보 출력
dat.describe() # 숫자 필드에 대한 간단한 요약통계량 출력

## pandas.DataFrame을 ndarray로부터 정의
dat2 = pd.DataFrame(np.random.randint(low = 1, high = 16, size = (4, 4)))
dat2
dat2.describe()

,field1,field2,field3
0,doc,0,alpha
1,dog,1,beta
2,dor,2,phi


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
field1    3 non-null object
field2    3 non-null int64
field3    3 non-null category
dtypes: category(1), int64(1), object(1)
memory usage: 235.0+ bytes


,field2
count,3.0
mean,1.0
std,1.0
min,0.0
25%,0.5
50%,1.0
75%,1.5
max,2.0


,0,1,2,3
0,3,9,12,14
1,11,11,6,1
2,10,5,5,14
3,3,7,12,6


,0,1,2,3
count,4.000000,4.000000,4.000000,4.000000
mean,6.750000,8.000000,8.750000,8.750000
std,4.349329,2.581989,3.774917,6.396614
min,3.000000,5.000000,5.000000,1.000000
25%,3.000000,6.500000,5.750000,4.750000
50%,6.500000,8.000000,9.000000,10.000000
75%,10.250000,9.500000,12.000000,14.000000
max,11.000000,11.000000,12.000000,14.000000


In [8]:
## pandas.DataFrame 정의의 잘못된 예
dat2 <- data.frame(field1 = c('doc', 'dog', 'dor'),
                  field2 = 1:2,
                  field3 = factor(c('alpha', 'beta', 'phi'), levels = c('phi', 'beta', 'alpha')))
dat2

SyntaxError: invalid syntax (<ipython-input-8-c9eadc5afc16>, line 3)

## 1. Reading data from external sources
+ 우리가 내려받는 데이터가 항상 '곧장 사용할 수 있는' 형태로 되어 있지는 않습니다.
+ 먼저 '깨끗한' 데이터를 불러오는 방법을 알아보고, 그렇지 않은 데이터를 읽어들이는 방법을 알아봅니다.
+ 데이터를 불러온 후에는 `.info` 메서드나 `.describe`, `.head`, `.shape` 등의 메서드를 이용해서 대략의 구조를 파악합니다.

### 1.1 Reading 'neat' data
+ `pandas` 패키지의 `read_csv` 함수: 쉼표로 구분된 파일을 읽어들입니다.
+ `pandas` 패키지의 `read_excel` 함수: xlsx 형식으로 된 엑셀 파일을 읽어들입니다.
+ 이외에 `pandas` 패키지의 `read_fwf` 함수는 고정폭 (fixed width) 파일을 읽어들입니다. (정부 마이크로데이터에서 주로 쓰임)

In [9]:
pla = pd.read_csv(filepath_or_buffer= 'C:/Users/sigma/Dropbox/Chores/2018S/GeoCONDA/W1/Plastic_trade.csv')
pla.head()
pla.describe()
pla.shape # 행과 열의 수

,Flow,Reporter,Partner,Netweight_kg,Value_usd,year,Reporter_nm,Partner_nm
0,Import,ALB,BGR,124190.0,107621.0,2010,ALBANIA,BULGARIA
1,Export,ALB,CHN,812861.0,425149.0,2010,ALBANIA,CHINA
2,Import,ALB,DEU,2347640.0,943464.0,2010,ALBANIA,GERMANY
3,Export,ALB,DEU,678160.0,349723.0,2010,ALBANIA,GERMANY
4,Import,ALB,GRC,1516306.0,647468.0,2010,ALBANIA,GREECE


,Netweight_kg,Value_usd,year
count,3.693900e+04,3.710000e+04,37100.000000
mean,6.420892e+06,3.133990e+06,2013.434852
std,6.761515e+07,3.030805e+07,2.230447
min,0.000000e+00,0.000000e+00,2010.000000
25%,1.744700e+04,9.243250e+03,2012.000000
50%,1.000000e+05,5.812200e+04,2013.000000
75%,7.514775e+05,4.004932e+05,2015.000000
max,4.069593e+09,1.203115e+09,2017.000000


(37100, 8)

__위에서 불러온 `pla` object는 이후에 계속 쓰입니다. __  

## 1.2 Reading 'dirty (or uncleaned)' data
- 일반적인 `data.frame` 형식으로 정리되지 않은 데이터를 불러오는 방법을 설명합니다.
- 통계청에서 내려받는 엑셀 파일들이 주로 정리되지 않은 데이터에 속합니다.
- 아래 코드 블록은 국민건강보험공단에서 내려받은 한 엑셀 데이터입니다. 위쪽에 메타데이터가 붙어 있어서, 별도의 처리 없이 불러오면 `DataFrame`이 뒤죽박죽이 되어 있는 모습을 볼 수 있습니다.
- `pandas.read_excel`의 `skiprows`와 `range` 인수는 각각 `integer`와 `character`형의 셀 범위를 받아서 엑셀 파일을 읽어옵니다. 적절한 인수값을 입력하면 불필요한 내용들의 영향을 없앨 수 있습니다.
    - `skiprows`는 몇 행을 무시할지를 나타냅니다.
    - `range`는 어느 열을 읽어들일지를 나타냅니다. 범위는 `,`와 `:`로 구분할 수 있습니다. 

In [10]:
# read_excel
atc = pd.read_excel(io = 'C:/Users/sigma/Dropbox/Chores/2018S/GeoCONDA/W1/ATC.xlsx', 
                    skiprows = 7,
                    range = 'A:CW')
atc.head()

,ATC코드,ATC코드명,시도명칭,시군구명칭,요양기관종별,수량,금액,수량.1,금액.1,수량.2,...,수량.43,금액.43,수량.44,금액.44,수량.45,금액.45,수량.46,금액.46,수량.47,금액.47
0,계,NaN,NaN,NaN,NaN,887561,121440459,895580,112871281,880231,...,974133,101890716,947695,99222749,906459,98026248,873271,88737443,1011224,103600877
1,B03A,IRON PREPARATIONS,부산,부산강서구,병원,3822,422748,2972,329892,1982,...,6,35514,28,14308,1,5919,31,8769,31,8769
2,B03A,IRON PREPARATIONS,부산,부산강서구,약국,3797,507801,3154,356346,3095,...,725,61035,568,49610,538,60560,1028,83620,593,66130
3,B03A,IRON PREPARATIONS,부산,부산강서구,의원,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,B03A,IRON PREPARATIONS,부산,부산금정구,병원,3567,655252,2917,598079,3137,...,3966,418784,4220,457487,4386,711677,3953,468468,4182,470108


## 2. Split-Apply-Combine

![](http://library.open.oregonstate.edu/computationalbiology/wp-content/uploads/sites/3/2016/10/III.8_50_dplyr_do.png)

## SAC: basic
+ `DataFrame`을 일정한 기준으로 나누고(split), 나눈 데이터에 각각 원하는 연산을 수행하고(apply), 얻은 결과를 합치는(combine)것을 말합니다. 대용량 데이터베이스에서 자료를 분산처리하는 원리와 유사하며, `DataFrame` 형태로 된 데이터를 정리하거나 집계하는 데 유용한 접근법입니다.
+ 위 그림은 어종별 중량 데이터를 종에 따라 분리하고, 평균과 표준편차를 각각 구하는 과정을 보여줍니다.
+ 이 원리 하에서, __우리는 항상 데이터를 나눌 기준에 주목__하고, 나눠진 각 데이터에 어떤 연산을 적용할 것인가에 유의해야 합니다. 
+ 아래에서는 어떤 함수가 어떠한 기능을 수행하는지 설명하겠습니다.

### Grouping: `groupby`
- `pandas.groupby`는 자료를 묶을 하위 기준을 지정하는 함수입니다.
- `groupby(groupvar)`: `groupvar`라는 필드의 고윳값 (unique values)을 기준으로 정보를 묶어서 처리하겠다는 의미입니다.
- 계획한 대로 자료를 다 묶어서 처리했으면 반드시 다음 파이프 연산자 뒤에 `ungroup` 함수를 적용해서 grouping을 해제해야 합니다. __(매우 중요!)__

### Data subsetting: `filter` and `[]`
- `[]` 안에 논리 조건을 부여하여 원하는 레코드를 추출하고자 할 때 사용합니다.

### Data aggregation: `agg`
- `pandas.agg`는 자료를 지정된 기준 필드의 값들에 따라서 요약하는 함수입니다. '기준 필드'를 필요로 하기 때문에 반드시 `groupby` 뒤에 와야 합니다. 세부 설정을 위해서는 `dictionary` 형태의 자료를 넣어 어느 필드에 어떤 연산을 수행할지 지정해야 합니다.

### All the way ahead: `dfply.'>>'`
- '크거나 같은(>)' 기호 두 개가 연달아 들어 있는 이 표식을 __'파이프 연산자 (pipe operator)'__라 합니다. 이 연산자는 `dfply` 패키지를 설치한 후 불러들이면 이용할 수 있습니다.
- 최초에 데이터를 투입한 다음, 적용할 함수들을 파이프 연산자로 연결하여 처리합니다.
- 파이프 연산자가 연결된 상태에서, 투입된 자료 자신을 온점 `.`으로 표현합니다. 명시적으로 처리하지 않으면 자료가 파이프 연산자 뒤 함수의 첫 번째 인자로 투입됩니다.
- 파이프 연산자의 장점으로는, 데이터 처리 과정을 순차적으로 설계하고 이해하는 데 편리하다는 점을 들 수 있습니다.

In [11]:
#### Example: pipe operator
sw = pd.read_csv('C:/Users/sigma/Dropbox/Chores/2018S/GeoCONDA/W1/starwars.csv')
sw.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species
0,Luke Skywalker,172.0,77.0,blond,fair,blue,19.0,male,Tatooine,Human
1,C-3PO,167.0,75.0,NaN,gold,yellow,112.0,NaN,Tatooine,Droid
2,R2-D2,96.0,32.0,NaN,"white, blue",red,33.0,NaN,Naboo,Droid
3,Darth Vader,202.0,136.0,none,white,yellow,41.9,male,Tatooine,Human
4,Leia Organa,150.0,49.0,brown,light,brown,19.0,female,Alderaan,Human


In [21]:
len(sw.groupby(['skin_color']).groups) # skin_color 필드 기준으로 구분지은 뒤 고윳값(group 기준)을 출력

# 종, 키, 질량 필드를 선택하고, 종에 따라 키와 질량의 평균 및 레코드 개수를 구한다.
sw.filter(['species', 'height', 'mass'], axis = 1).groupby(['species']).agg({'height': [len, np.mean], 'mass': np.mean}).head()
swfilt = sw.filter(['species', 'height', 'mass'], axis = 1).groupby(['species']).agg({'height': [len, np.mean], 'mass': np.mean})
swfilt.columns = swfilt.columns.droplevel(0)
swfilt.head()
swfilt['mean']

sw.groupby('species', as_index=False).agg(lambda x: np.mean(x))

# filter 함수에서 axis 인수는 어느 방향으로 filtering할 것인가를 의미 (0: 행, 1: 열)
# 종과 출신지에 따라 묶은 뒤 레코드 개수 (이름의 길이)를 구한다.
sw.groupby(['species', 'homeworld']).agg({'name': len}).head()

31

height          mass
            len   mean   mean
species                      
Aleena      1.0   79.0   15.0
Besalisk    1.0  198.0  102.0
Cerean      1.0  198.0   82.0
Chagrian    1.0  196.0    NaN
Clawdite    1.0  168.0   55.0

,len,mean,mean
species,,,
Aleena,1.0,79.0,15.0
Besalisk,1.0,198.0,102.0
Cerean,1.0,198.0,82.0
Chagrian,1.0,196.0,NaN
Clawdite,1.0,168.0,55.0


,mean,mean
species,,
Aleena,79.000000,15.000000
Besalisk,198.000000,102.000000
Cerean,198.000000,82.000000
Chagrian,196.000000,NaN
Clawdite,168.000000,55.000000
Droid,140.000000,69.750000
Dug,112.000000,40.000000
Ewok,88.000000,20.000000
Geonosian,183.000000,80.000000


,species,height,mass,birth_year
0,Aleena,79.000000,15.000000,NaN
1,Besalisk,198.000000,102.000000,NaN
2,Cerean,198.000000,82.000000,92.000000
3,Chagrian,196.000000,NaN,NaN
4,Clawdite,168.000000,55.000000,NaN
5,Droid,140.000000,69.750000,53.333333
6,Dug,112.000000,40.000000,NaN
7,Ewok,88.000000,20.000000,8.000000
8,Geonosian,183.000000,80.000000,NaN
9,Gungan,208.666667,74.000000,52.000000


,,name
species,homeworld,
Aleena,Aleen Minor,1
Besalisk,Ojom,1
Cerean,Cerea,1
Chagrian,Champala,1
Clawdite,Zolan,1


## Data transformation
### Long/wide data frame
- 분류 속성을 어떻게 처리하는가에 따른 `DataFrame`의 분류입니다.
    - long format: 분류 속성이 하나의 필드에 정리되어 세로로 긴 형태의 `DataFrame`
    - wide format: 분류 속성이 각 필드에 정리되어 가로로 긴 형태의 `DataFrame`
    - 우리는 앞으로 wide data frame을 long data frame으로 자주 바꾸게 됩니다.
- 아래 코드 블록은 Star Wars의 인물들을 성별과 눈동자 색에 따라 키와 몸무게의 평균이 어떠한지를 요약합니다. 분류 속성은 `gender`와 `eye_color`입니다.
- 정리된 내용을 각각 `gender`의 속성과 `eye_color`의 속성에 따라서 `height`와 `mass`를 나타내도록, 즉 wide format으로 바꾸어 보겠습니다.
- 아래 예시를 잘 활용하면 분류 속성이 여러 차원일 때, 데이터를 효과적으로 요약하여 제시할 수 있습니다.

In [117]:
sw_summary = sw.groupby(['gender', 'eye_color']).agg({'height': np.mean, 'mass': np.mean})
type(sw_summary)
sw_summary.head()
sw_summary.unstack()

pandas.core.frame.DataFrame

height  mass
gender eye_color              
female black       195.5  57.0
       blue        167.0  57.8
       brown       160.0  47.0
       hazel       178.0  55.0
       red, blue    96.0   NaN

height                                                         \
eye_color      black        blue blue-gray  brown dark   gold green, yellow   
gender                                                                        
female         195.5  167.000000       NaN  160.0  NaN    NaN           NaN   
hermaphrodite    NaN         NaN       NaN    NaN  NaN    NaN           NaN   
male           182.0  189.230769     182.0  168.5  NaN  191.0         216.0   
none             NaN         NaN       NaN    NaN  NaN    NaN           NaN   

                                           ...     mass                      \
eye_color      hazel      orange   pink    ...     gold green, yellow hazel   
gender                                     ...                                
female         178.0         NaN    NaN    ...      NaN           NaN  55.0   
hermaphrodite    NaN  175.000000    NaN    ...      NaN           NaN   NaN   
male           170.0  181.285714  180.0    ...      NaN         159.0  77.0   
none             NaN         NaN    NaN    ...      NaN           NaN   NaN   

                                                                      
eye_color      orange pink    red red, blue unknown white     yellow  
gender                                                                
female            NaN  NaN    NaN       NaN     NaN  48.0  55.000000  
hermaphrodite  1358.0  NaN    NaN       NaN     NaN   NaN        NaN  
male             67.2  NaN  101.5       NaN    31.5   NaN  85.714286  
none              NaN  NaN  140.0       NaN     NaN   NaN        NaN  

[4 rows x 30 columns]

In [33]:
starwars %>% group_by(gender, eye_color) %>% 
    summarize_at(.vars = vars(height, mass), .funs = funs(mean(., na.rm = TRUE))) %>% ungroup -> sw_summary

## gather height and mass
sw_summary %>% gather(key = vclass, value = val, 3:4) %>% # gather height and mass (the third and fourth column, respectively)
# fill values to val column, and fill the class names ('height' and 'mass') in vclass column
    mutate(agc1 = paste(gender, vclass, sep = '_'),
           agc2 = paste(eye_color, vclass, sep = '_')) -> sw_summary_wide

sw_summary_wide %>% dplyr::select(eye_color, agc1, val) %>% spread(key = agc1, value = val) %>% head(5)
sw_summary_wide %>% dplyr::select(gender, agc2, val) %>% spread(key = agc2, value = val) %>% head(5)

eye_color,female_height,female_mass,hermaphrodite_height,hermaphrodite_mass,male_height,male_mass,NA_height,NA_mass,none_height,none_mass
black,195.5,57.0,NA,NA,182.0000,79.50000,NA,NA,NaN,NaN
blue,167.0,57.8,NA,NA,189.2308,100.87500,NA,NA,NA,NA
blue-gray,NA,NA,NA,NA,182.0000,77.00000,NA,NA,NA,NA
brown,160.0,47.0,NA,NA,168.5000,69.56364,NA,NA,NA,NA
dark,NA,NA,NA,NA,NaN,NaN,NA,NA,NA,NA


gender,black_height,black_mass,blue-gray_height,blue-gray_mass,blue_height,blue_mass,brown_height,brown_mass,dark_height,...,"red, blue_height","red, blue_mass",red_height,red_mass,unknown_height,unknown_mass,white_height,white_mass,yellow_height,yellow_mass
female,195.5,57.0,NA,NA,167.0000,57.800,160.0,47.00000,NA,...,96,NaN,NA,NA,NaN,NaN,178,48,168.0000,55.00000
hermaphrodite,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
male,182.0,79.5,182,77,189.2308,100.875,168.5,69.56364,NaN,...,NA,NA,190.5,101.5,136,31.5,NA,NA,180.1111,85.71429
none,NaN,NaN,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,200.0,140.0,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,96.5,32.0,NA,NA,NA,NA,167.0000,75.00000


## 3. Real-world example: Recycling crisis

4월 초 며칠 동안, 일명 '재활용 대란'이 있었습니다. 아시다시피, 원인으로 국제 폐비닐/폐플라스틱 시세 하락으로 인한 중국으로의 수출 중단이 원인으로 지목되었습니다. 여기서, 우리는 실제 시세 하락은 얼마나 일어났는가라는 질문을 제기할 수 있습니다. 우리는 데이터를 통해서 간접적으로 그 질문을 풀어보려 합니다. 자료는 [UN Comtrade](https://comtrade.un.org/)에서 2010년에서 2017년 간의 세계 국가별 무역데이터를 내려받았고 (학내에서만 무료로 내려받을 수 있음), 미리 폐플라스틱 (HS code: 9315)에 해당하는 상품코드에 맞추어 연도별 데이터를 추출하였습니다. 과제는 단일 연도의 전체 데이터를 대상으로 수행하시게 될 것입니다.
이 데이터는 보고국(reporter)과 대상국(partner) 명칭, 수출/입 여부, 폐플라스틱 순교역량(kg), 교역액(USD) 정보, 통계연도를 담고 있습니다. 우리는 이 파일에서 몇 가지 질문에 대한 해답을 얻어내고자 합니다.

1. 지난 8년 간 세계 폐플라스틱 무역 총량을 알아봅니다.
2. 지난 8년 간 국가별 폐플라스틱 무역을 결산해봅니다.
3. 지난 8년 간 국가별 대중국 폐플라스틱 무역 총량을 알아봅니다.
4. 한국의 폐플라스틱 무역의 시계열을 액수와 중량에 따라 조사해 봅니다.
5. 지난 8년 간 세계 폐플라스틱 시세 변동을 알아봅니다.
    - 단, '세계 폐플라스틱 시세'를 데이터로부터 살펴보기 위해서 다음과 같이 조작적으로 정의합시다: '세계 폐플라스틱 시세'란 단위순중량(kg)당 교역액(USD)를 의미합니다. 폐플라스틱의 세부구분은 고려하지 않습니다.

### 3.1
- 자료에는 '세계 전체'를 아우르는 정보가 없으므로, 정보를 생성한 다음 집계하도록 합시다.
- 아래 첫 번째 코드 블록에서는 NA값을 빼고 순중량과 교역액을 합산했습니다. 이 때 제외된 값들이 무엇이었느냐에 따라서 왜곡이 발생할 수 있습니다. 두 번째 코드 블록에서 확인해 보겠습니다.
- 두 번째 코드 블록에서 `sort`에서 필드 이름을 문자열로 정의한 후에 `ascending`을 `False`로 처리하면 '내림차순 정렬'을 하라는 뜻입니다. 확인 결과, 순중량이 포함되지 않은 사례들이 161건 있었고, 교역액 필드를 볼 때에 교역량이 큰 사례들에서 중량이 누락되어 있으므로, 이들을 제외하고 합산한 결과에 상당한 왜곡이 있으리라고 예상할 수 있습니다. 특히 2016년 사례가 그렇습니다.
- 위의 문제점에 착안해서, 세 번째 코드 블록에서는 어느 한 필드에서도 `NaN`값이 존재하지 않는 레코드들만 추출한 후 같은 분석을 수행해 봅니다.
    - `NaN`값을 제외하기 위하여 `numpy.isnan` 함수를 사용했습니다. 사용례에 유의하십시오.

In [19]:
pla = pd.read_csv('C:/Users/sigma/Dropbox/Chores/2018S/GeoCONDA/W1/Plastic_trade.csv')
pla.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37100 entries, 0 to 37099
Data columns (total 8 columns):
Flow            37100 non-null object
Reporter        37100 non-null object
Partner         37100 non-null object
Netweight_kg    36939 non-null float64
Value_usd       37100 non-null float64
year            37100 non-null int64
Reporter_nm     37100 non-null object
Partner_nm      37097 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 2.3+ MB


In [22]:
pla['all'] = 'World' # all이라는 필드 생성 후 값을 'World'로 할당
pla.groupby(['all']).agg({'Netweight_kg': np.sum, 'Value_usd': np.sum}) # aggregate 함수에서 각 항목을 dictionary 형태로 정의하고 있습니다. 유의하여 사용하십시오.
pla.groupby(['all', 'year']).agg({'Netweight_kg': np.sum, 'Value_usd': np.sum})
pla.groupby(['all', 'year', 'Flow']).agg({'Netweight_kg': np.sum, 'Value_usd': np.sum})
pla.groupby(['all', 'year', 'Flow']).agg({'Netweight_kg': np.sum, 'Value_usd': np.sum}).unstack('year') # year만 group 해제하여 가로로 긴 DataFrame으로 만듦

pla.groupby(['all', 'year', 'Flow'], as_index=False).agg(lambda x: np.mean(x))


,Netweight_kg,Value_usd
all,,
World,2.371813e+11,1.162710e+11


Netweight_kg     Value_usd
all   year                            
World 2010  3.384326e+10  1.490190e+10
      2011  3.345442e+10  1.750396e+10
      2012  3.375576e+10  1.752888e+10
      2013  2.896921e+10  1.658599e+10
      2014  3.405141e+10  1.714635e+10
      2015  3.189631e+10  1.358173e+10
      2016  2.659444e+10  1.245763e+10
      2017  1.461652e+10  6.564595e+09

Netweight_kg     Value_usd
all   year Flow                                 
World 2010 Export     1.533793e+10  6.055398e+09
           Import     1.578823e+10  8.067401e+09
           Re-Export  2.715697e+09  7.783281e+08
           Re-Import  1.388963e+06  7.777640e+05
      2011 Export     1.482272e+10  7.024458e+09
           Import     1.597427e+10  9.618217e+09
           Re-Export  2.655678e+09  8.600322e+08
           Re-Import  1.748614e+06  1.257839e+06
      2012 Export     1.484154e+10  6.983048e+09
           Import     1.594007e+10  9.573884e+09
           Re-Export  2.972745e+09  9.708963e+08
           Re-Import  1.407305e+06  1.055318e+06
      2013 Export     1.288642e+10  6.553549e+09
           Import     1.475308e+10  9.299623e+09
           Re-Export  1.328605e+09  7.322756e+08
           Re-Import  1.106226e+06  5.418060e+05
      2014 Export     1.538328e+10  6.779584e+09
           Import     1.568936e+10  9.447375e+09
           Re-Export  2.976671e+09  9.180063e+08
           Re-Import  2.096504e+06  1.385985e+06
      2015 Export     1.425913e+10  5.630390e+09
           Import     1.486857e+10  7.065581e+09
           Re-Export  2.761138e+09  8.830854e+08
           Re-Import  7.473772e+06  2.670648e+06
      2016 Export     1.132727e+10  5.132261e+09
           Import     1.524300e+10  6.464587e+09
           Re-Export  2.094125e+07  8.597216e+08
           Re-Import  3.231188e+06  1.055903e+06
      2017 Export     8.711768e+09  3.742008e+09
           Import     5.887667e+09  2.274660e+09
           Re-Export  1.237663e+07  5.460998e+08
           Re-Import  4.704120e+06  1.828282e+06

Netweight_kg                                            \
year                     2010          2011          2012          2013   
all   Flow                                                                
World Export     1.533793e+10  1.482272e+10  1.484154e+10  1.288642e+10   
      Import     1.578823e+10  1.597427e+10  1.594007e+10  1.475308e+10   
      Re-Export  2.715697e+09  2.655678e+09  2.972745e+09  1.328605e+09   
      Re-Import  1.388963e+06  1.748614e+06  1.407305e+06  1.106226e+06   

                                                                         \
year                     2014          2015          2016          2017   
all   Flow                                                                
World Export     1.538328e+10  1.425913e+10  1.132727e+10  8.711768e+09   
      Import     1.568936e+10  1.486857e+10  1.524300e+10  5.887667e+09   
      Re-Export  2.976671e+09  2.761138e+09  2.094125e+07  1.237663e+07   
      Re-Import  2.096504e+06  7.473772e+06  3.231188e+06  4.704120e+06   

                    Value_usd                                            \
year                     2010          2011          2012          2013   
all   Flow                                                                
World Export     6.055398e+09  7.024458e+09  6.983048e+09  6.553549e+09   
      Import     8.067401e+09  9.618217e+09  9.573884e+09  9.299623e+09   
      Re-Export  7.783281e+08  8.600322e+08  9.708963e+08  7.322756e+08   
      Re-Import  7.777640e+05  1.257839e+06  1.055318e+06  5.418060e+05   

                                                                         
year                     2014          2015          2016          2017  
all   Flow                                                               
World Export     6.779584e+09  5.630390e+09  5.132261e+09  3.742008e+09  
      Import     9.447375e+09  7.065581e+09  6.464587e+09  2.274660e+09  
      Re-Export  9.180063e+08  8.830854e+08  8.597216e+08  5.460998e+08  
      Re-Import  1.385985e+06  2.670648e+06  1.055903e+06  1.828282e+06

,all,year,Flow,Netweight_kg,Value_usd
0,World,2010,Export,6.930834e+06,2.736285e+06
1,World,2010,Import,7.299230e+06,3.724562e+06
2,World,2010,Re-Export,2.207884e+07,6.327871e+06
3,World,2010,Re-Import,1.543292e+05,8.641822e+04
4,World,2011,Export,6.302176e+06,2.986589e+06
5,World,2011,Import,6.993989e+06,4.203766e+06
6,World,2011,Re-Export,2.505357e+07,8.113511e+06
7,World,2011,Re-Import,1.165743e+05,8.385593e+04
8,World,2012,Export,6.509447e+06,3.034788e+06
9,World,2012,Import,6.903453e+06,4.060171e+06


In [141]:
pla[-np.isnan(pla.Netweight_kg)].head(10)

,Flow,Reporter,Partner,Netweight_kg,Value_usd,year,Reporter_nm,Partner_nm,all
0,Import,ALB,BGR,124190.0,107621.0,2010,ALBANIA,BULGARIA,World
1,Export,ALB,CHN,812861.0,425149.0,2010,ALBANIA,CHINA,World
2,Import,ALB,DEU,2347640.0,943464.0,2010,ALBANIA,GERMANY,World
3,Export,ALB,DEU,678160.0,349723.0,2010,ALBANIA,GERMANY,World
4,Import,ALB,GRC,1516306.0,647468.0,2010,ALBANIA,GREECE,World
5,Import,ALB,ITA,65626.0,29150.0,2010,ALBANIA,ITALY,World
6,Export,ALB,ITA,18088.0,18117.0,2010,ALBANIA,ITALY,World
7,Import,ALB,SRB,15300.0,5519.0,2010,ALBANIA,SERBIA,World
8,Export,ALB,SRB,1500.0,10040.0,2010,ALBANIA,SERBIA,World
9,Export,ALB,SVN,21000.0,13204.0,2010,ALBANIA,SLOVENIA,World


In [139]:
pla_nw = pla[-np.isnan(pla.Netweight_kg)]
pla_nw.sort_values('Value_usd', ascending = False).head(10) # sort에는 sort_values와 sort_index가 있음; ascending 인수는 오름차순 정렬 또는 내림차순 정렬을 의미
pla_nw.sort_values('Value_usd', ascending = False).shape
pla[-np.isnan(pla.Value_usd)].sort_values('Netweight_kg', ascending = False).head(10)

,Flow,Reporter,Partner,Netweight_kg,Value_usd,year,Reporter_nm,Partner_nm,all
695,Import,CHN,HKG,2.009524e+09,1.203115e+09,2010,CHINA,HONG KONG,World
1543,Export,HKG,CHN,4.069593e+09,1.180614e+09,2010,HONG KONG,CHINA,World
6152,Export,HKG,CHN,3.314772e+09,1.049255e+09,2011,HONG KONG,CHINA,World
10784,Export,HKG,CHN,3.160473e+09,1.008721e+09,2012,HONG KONG,CHINA,World
10785,Re-Export,HKG,CHN,2.856332e+09,9.161063e+08,2012,HONG KONG,CHINA,World
5238,Import,CHN,HKG,1.323168e+09,9.091630e+08,2011,CHINA,HONG KONG,World
20454,Export,HKG,CHN,2.990696e+09,9.076048e+08,2014,HONG KONG,CHINA,World
20455,Re-Export,HKG,CHN,2.910714e+09,8.879360e+08,2014,HONG KONG,CHINA,World
25278,Export,HKG,CHN,2.797668e+09,8.850445e+08,2015,HONG KONG,CHINA,World
25279,Re-Export,HKG,CHN,2.722084e+09,8.628934e+08,2015,HONG KONG,CHINA,World


(36939, 9)

,Flow,Reporter,Partner,Netweight_kg,Value_usd,year,Reporter_nm,Partner_nm,all
1543,Export,HKG,CHN,4.069593e+09,1.180614e+09,2010,HONG KONG,CHINA,World
6152,Export,HKG,CHN,3.314772e+09,1.049255e+09,2011,HONG KONG,CHINA,World
10784,Export,HKG,CHN,3.160473e+09,1.008721e+09,2012,HONG KONG,CHINA,World
20454,Export,HKG,CHN,2.990696e+09,9.076048e+08,2014,HONG KONG,CHINA,World
20455,Re-Export,HKG,CHN,2.910714e+09,8.879360e+08,2014,HONG KONG,CHINA,World
10785,Re-Export,HKG,CHN,2.856332e+09,9.161063e+08,2012,HONG KONG,CHINA,World
25278,Export,HKG,CHN,2.797668e+09,8.850445e+08,2015,HONG KONG,CHINA,World
25279,Re-Export,HKG,CHN,2.722084e+09,8.628934e+08,2015,HONG KONG,CHINA,World
1544,Re-Export,HKG,CHN,2.534989e+09,7.233050e+08,2010,HONG KONG,CHINA,World
6153,Re-Export,HKG,CHN,2.467585e+09,7.890027e+08,2011,HONG KONG,CHINA,World


In [146]:
pla_wc = pla[-np.isnan(pla.Netweight_kg) & -np.isnan(pla.Value_usd)]
pla_wcsum = pla_wc.groupby('all').agg({'Netweight_kg': np.sum,
                                       'Value_usd': np.sum})
pla_wcysum = pla_wc.groupby(['all', 'year']).agg({'Netweight_kg': np.sum,
                                                  'Value_usd': np.sum})
pla_wcyfsum = pla_wc.groupby(['all', 'year', 'Flow']).agg({'Netweight_kg': np.sum,
                                                           'Value_usd': np.sum})

pla_wcsum
pla_wcysum
pla_wcyfsum

,Netweight_kg,Value_usd
all,,
World,2.371813e+11,1.144038e+11


Netweight_kg     Value_usd
all   year                            
World 2010  3.384326e+10  1.490190e+10
      2011  3.345442e+10  1.750396e+10
      2012  3.375576e+10  1.741021e+10
      2013  2.896921e+10  1.658599e+10
      2014  3.405141e+10  1.714635e+10
      2015  3.189631e+10  1.358171e+10
      2016  2.659444e+10  1.073311e+10
      2017  1.461652e+10  6.540611e+09

Netweight_kg     Value_usd
all   year Flow                                 
World 2010 Export     1.533793e+10  6.055398e+09
           Import     1.578823e+10  8.067396e+09
           Re-Export  2.715697e+09  7.783281e+08
           Re-Import  1.388963e+06  7.777640e+05
      2011 Export     1.482272e+10  7.024458e+09
           Import     1.597427e+10  9.618216e+09
           Re-Export  2.655678e+09  8.600322e+08
           Re-Import  1.748614e+06  1.257839e+06
      2012 Export     1.484154e+10  6.891275e+09
           Import     1.594007e+10  9.546980e+09
           Re-Export  2.972745e+09  9.708963e+08
           Re-Import  1.407305e+06  1.055318e+06
      2013 Export     1.288642e+10  6.553549e+09
           Import     1.475308e+10  9.299623e+09
           Re-Export  1.328605e+09  7.322756e+08
           Re-Import  1.106226e+06  5.418060e+05
      2014 Export     1.538328e+10  6.779584e+09
           Import     1.568936e+10  9.447375e+09
           Re-Export  2.976671e+09  9.180063e+08
           Re-Import  2.096504e+06  1.385985e+06
      2015 Export     1.425913e+10  5.630369e+09
           Import     1.486857e+10  7.065580e+09
           Re-Export  2.761138e+09  8.830854e+08
           Re-Import  7.473772e+06  2.670648e+06
      2016 Export     1.132727e+10  4.257018e+09
           Import     1.524300e+10  6.464519e+09
           Re-Export  2.094125e+07  1.052172e+07
           Re-Import  3.231188e+06  1.055903e+06
      2017 Export     8.711768e+09  3.729030e+09
           Import     5.887667e+09  2.263661e+09
           Re-Export  1.237663e+07  5.460914e+08
           Re-Import  4.704120e+06  1.828282e+06

### 3.2 

In [156]:
pla_wccf = pla_wc.groupby(['Reporter', 'Reporter_nm', 'Flow']).agg({'Netweight_kg': np.sum, 'Value_usd': np.sum})
pla_wccf.head(12)
pla_wccfc = pla_wccf.reset_index() # groupby 해제
pla_wccfc[pla_wccfc.Flow == 'Import'].sort_values('Value_usd', ascending = False).head(5)
pla_wccfc[pla_wccfc.Flow == 'Export'].sort_values('Value_usd', ascending = False).head(5)
pla_wccfc[pla_wccfc.Flow == 'Re-Import'].sort_values('Value_usd', ascending = False).head(5)
pla_wccfc[pla_wccfc.Flow == 'Re-Export'].sort_values('Value_usd', ascending = False).head(5)

Netweight_kg    Value_usd
Reporter Reporter_nm          Flow                                
ABW      ARUBA                Import            419.0       1801.0
AGO      ANGOLA               Import        1670777.0    3249450.0
ALB      ALBANIA              Export       19885126.0   13202636.0
                              Import       14088276.0    5680646.0
AND      ANDORRA              Export        7575360.0    1085887.0
                              Import         106457.0      10315.0
ARE      UNITED ARAB EMIRATES Export      225834713.0   69352245.0
                              Import       52858666.0   35525623.0
                              Re-Export     9581072.0    3980199.0
ARG      ARGENTINA            Export      153709866.0  115702253.0
                              Import         994772.0    1155236.0
ARM      ARMENIA              Export         963244.0     538501.0

,Reporter,Reporter_nm,Flow,Netweight_kg,Value_usd
75,CHN,CHINA,Import,5.435407e+10,3.633695e+10
153,HKG,HONG KONG,Import,2.475193e+10,9.466704e+09
388,USA,UNITED STATES,Import,3.127863e+09,1.909131e+09
281,NLD,NETHERLANDS,Import,3.501518e+09,1.484676e+09
98,DEU,GERMANY,Import,3.531763e+09,1.453888e+09


,Reporter,Reporter_nm,Flow,Netweight_kg,Value_usd
387,USA,UNITED STATES,Export,1.552529e+10,6.865813e+09
152,HKG,HONG KONG,Export,1.826887e+10,6.526781e+09
185,JPN,JAPAN,Export,1.215760e+10,5.730225e+09
97,DEU,GERMANY,Export,1.118274e+10,4.824569e+09
128,GBR,UNITED KINGDOM,Export,6.143680e+09,2.029234e+09


,Reporter,Reporter_nm,Flow,Netweight_kg,Value_usd
296,PAK,PAKISTAN,Re-Import,10599464.0,2833013.0
68,CAN,CANADA,Re-Import,4545031.0,2580185.0
126,FRA,FRANCE,Re-Import,1856898.0,1546052.0
327,RUS,RUSSIAN FEDERATION,Re-Import,2238450.0,1150009.0
313,PRT,PORTUGAL,Re-Import,969413.0,785876.0


,Reporter,Reporter_nm,Flow,Netweight_kg,Value_usd
154,HKG,HONG KONG,Re-Export,1.517346e+10,5.598515e+09
389,USA,UNITED STATES,Re-Export,1.429108e+08,5.766433e+07
332,SAU,SAUDI ARABIA,Re-Export,9.510300e+07,2.770057e+07
67,CAN,CANADA,Re-Export,6.707501e+06,5.563311e+06
8,ARE,UNITED ARAB EMIRATES,Re-Export,9.581072e+06,3.980199e+06


### 3.3
- 아래 코드 블록은 대상국이 중국 (ISO 코드가 'CHN'으로 된)인 자료를 대상으로 각각 순중량과 교역액의 연도별 변화추이를 정리하는 내용입니다. 코드의 흐름을 잘 관찰해 보시기 바랍니다.
- 중간에 `dplyr::select`로 표기된 부분은 채워넣을 값을 각각 순중량과 교역액으로 한정하는 역할을 합니다.

In [164]:
pla_wcv_wide = pla_wc[pla_wc.Partner == 'CHN'].groupby(['Reporter', 'Reporter_nm', 'Flow', 'year']).agg(
    {'Netweight_kg': np.sum,
     'Value_usd': np.sum}).filter(like = 'Value_usd', axis = 1).unstack('year')

pla_wcw_wide = pla_wc[pla_wc.Partner == 'CHN'].groupby(['Reporter', 'Reporter_nm', 'Flow', 'year']).agg(
    {'Netweight_kg': np.sum,
     'Value_usd': np.sum}).filter(like = 'Netweight_kg', axis = 1).unstack('year')

pla_wcw_wide.head(12)
pla_wcv_wide.head(12)

Netweight_kg                          \
year                                            2010        2011        2012   
Reporter Reporter_nm          Flow                                             
AGO      ANGOLA               Import        103322.0      8511.0       803.0   
ALB      ALBANIA              Export        812861.0    925531.0    794717.0   
                              Import             NaN         NaN         NaN   
ARE      UNITED ARAB EMIRATES Export       6955883.0  11371547.0  13936582.0   
                              Import        637212.0   1124450.0   1864281.0   
                              Re-Export    1721050.0    134500.0    714308.0   
ARG      ARGENTINA            Export      25904750.0  25288490.0  21129471.0   
                              Import             NaN         NaN         NaN   
ARM      ARMENIA              Export        457437.0         NaN         NaN   
                              Import             NaN         NaN         NaN   
ATG      ANTIGUA AND BARBUDA  Export         18700.0     58896.0         NaN   
                              Re-Export      18700.0     58896.0         NaN   

                                                                             \
year                                           2013        2014        2015   
Reporter Reporter_nm          Flow                                            
AGO      ANGOLA               Import        41553.0      2468.0     13927.0   
ALB      ALBANIA              Export      1398855.0         NaN         NaN   
                              Import        18720.0         NaN         NaN   
ARE      UNITED ARAB EMIRATES Export     49877369.0  15891719.0  12303725.0   
                              Import      2221856.0   3374501.0   3157278.0   
                              Re-Export    134810.0    313955.0    106530.0   
ARG      ARGENTINA            Export     14917322.0   8540762.0   7674051.0   
                              Import            NaN         NaN         NaN   
ARM      ARMENIA              Export            NaN         NaN         NaN   
                              Import            NaN         2.0         NaN   
ATG      ANTIGUA AND BARBUDA  Export        22000.0         NaN     42960.0   
                              Re-Export         NaN         NaN         NaN   

                                                               
year                                          2016       2017  
Reporter Reporter_nm          Flow                             
AGO      ANGOLA               Import           NaN        NaN  
ALB      ALBANIA              Export           NaN        NaN  
                              Import           NaN        NaN  
ARE      UNITED ARAB EMIRATES Export     8617665.0        NaN  
                              Import     2143907.0        NaN  
                              Re-Export        NaN        NaN  
ARG      ARGENTINA            Export     3269874.0  1712664.0  
                              Import           NaN    91610.0  
ARM      ARMENIA              Export           NaN        NaN  
                              Import        9450.0        NaN  
ATG      ANTIGUA AND BARBUDA  Export       20000.0        NaN  
                              Re-Export        NaN        NaN

Value_usd                          \
year                                           2010        2011        2012   
Reporter Reporter_nm          Flow                                            
AGO      ANGOLA               Import       104766.0      7180.0       378.0   
ALB      ALBANIA              Export       425149.0    619139.0    565976.0   
                              Import            NaN         NaN         NaN   
ARE      UNITED ARAB EMIRATES Export      2071036.0   3991057.0   4889397.0   
                              Import       694508.0    575945.0   1139183.0   
                              Re-Export    413230.0     29866.0    293564.0   
ARG      ARGENTINA            Export     17224578.0  21495661.0  17552048.0   
                              Import            NaN         NaN         NaN   
ARM      ARMENIA              Export       353684.0         NaN         NaN   
                              Import            NaN         NaN         NaN   
ATG      ANTIGUA AND BARBUDA  Export         7407.0     20370.0         NaN   
                              Re-Export      7407.0     20370.0         NaN   

                                                                           \
year                                           2013       2014       2015   
Reporter Reporter_nm          Flow                                          
AGO      ANGOLA               Import       109724.0     7200.0    28216.0   
ALB      ALBANIA              Export       962869.0        NaN        NaN   
                              Import        13482.0        NaN        NaN   
ARE      UNITED ARAB EMIRATES Export      4501610.0  4985469.0  4330899.0   
                              Import      1224180.0  2123569.0  2010681.0   
                              Re-Export     40669.0   103677.0    68523.0   
ARG      ARGENTINA            Export     10238801.0  4051357.0  2991986.0   
                              Import            NaN        NaN        NaN   
ARM      ARMENIA              Export            NaN        NaN        NaN   
                              Import            NaN      176.0        NaN   
ATG      ANTIGUA AND BARBUDA  Export         7407.0        NaN     9815.0   
                              Re-Export         NaN        NaN        NaN   

                                                              
year                                          2016      2017  
Reporter Reporter_nm          Flow                            
AGO      ANGOLA               Import           NaN       NaN  
ALB      ALBANIA              Export           NaN       NaN  
                              Import           NaN       NaN  
ARE      UNITED ARAB EMIRATES Export     2852756.0       NaN  
                              Import     1114514.0       NaN  
                              Re-Export        NaN       NaN  
ARG      ARGENTINA            Export     1242567.0  748326.0  
                              Import           NaN   28632.0  
ARM      ARMENIA              Export           NaN       NaN  
                              Import        3781.0       NaN  
ATG      ANTIGUA AND BARBUDA  Export        4444.0       NaN  
                              Re-Export        NaN       NaN

### 3.4
- 3.3에서 연도를 분류 속성으로 한 wide format data frame을 생성했으므로, 시계열 탐색은 해당 data.frame으로부터 `filter` 함수를 적용하여 탐색해 봅니다.
- 아래 결과를 보면, 한국의 대중국 폐플라스틱 무역은 재수출이나 재수입이 없는 수출과 수입만으로 이루어져 있음을 알 수 있고, 수출순중량에 다소 등락이 있는 반면 수출액은 경향적인 감소 추세를 보이고 있음을 알 수 있습니다.

In [167]:
pla_wcw_wide = pla_wcw_wide.reset_index()
pla_wcv_wide = pla_wcv_wide.reset_index()
pla_wcw_wide[pla_wcw_wide.Reporter == 'KOR']
pla_wcv_wide[pla_wcv_wide.Reporter == 'KOR']

Reporter         Reporter_nm    Flow Netweight_kg               \
year                                              2010         2011   
147       KOR  KOREA, REPUBLIC OF  Export  171802127.0  141239829.0   
148       KOR  KOREA, REPUBLIC OF  Import    3652620.0    2884578.0   

                                                                       \
year         2012         2013         2014         2015         2016   
147   157891729.0  154110543.0  146778772.0  148748722.0  151366129.0   
148     1538283.0    1859189.0    2623824.0    2379751.0     842278.0   

                   
year         2017  
147   119574904.0  
148     1799405.0

Reporter         Reporter_nm    Flow   Value_usd                          \
year                                             2010        2011        2012   
147       KOR  KOREA, REPUBLIC OF  Export  55567324.0  50194885.0  51194379.0   
148       KOR  KOREA, REPUBLIC OF  Import   2459961.0   2893720.0   1128061.0   

                                                                  
year        2013        2014        2015        2016        2017  
147   56344812.0  47579141.0  43200394.0  39844456.0  25191637.0  
148    1011959.0   1420037.0   1026864.0    440171.0   2621146.0

### 3.5
- 3.1에서 만든 표로부터 메트릭 톤당 '수입가격'과 '수출가격'을 계산해 보겠습니다.
- 참고: 수입가격은 물품구입대금에 관세, 운송비, 금융비용 등이 더해지므로 수출가격보다 높은 경향이 있습니다. 수입가격이 높다는 것은, 그 배경에 있는 국내가격이 높거나 제품 가공 후 재수출가격이 높음을 뜻합니다.
- 그런데 2017년에 평균 수출가격이 평균 수입가격을 앞서는 현상이 나타납니다. 이는 나머지 7년 간의 패턴과 상반된 것으로, 수입국의 수입유인이 매우 낮아졌음을 의미합니다. 이로부터 4월의 재활용 대란의 배경이 어느 정도 설명됩니다.
- 더 깊은 이야기는 원료인 석유류 및 석유화학제품의 국제무역 자료 분석, 그리고 각국의 플라스틱 재생 업계의 업황 분석을 통해서 이어갈 수 있을 것입니다.

In [172]:
pla_wcyfsum['prunit'] = 1000 * pla_wcyfsum.Value_usd / pla_wcyfsum.Netweight_kg
pla_wcyfsum.head(10)

pla_wcyfsum.unstack('year').filter(like = 'prunit', axis = 1)

Netweight_kg     Value_usd      prunit
all   year Flow                                             
World 2010 Export     1.533793e+10  6.055398e+09  394.798785
           Import     1.578823e+10  8.067396e+09  510.975189
           Re-Export  2.715697e+09  7.783281e+08  286.603442
           Re-Import  1.388963e+06  7.777640e+05  559.960201
      2011 Export     1.482272e+10  7.024458e+09  473.898135
           Import     1.597427e+10  9.618216e+09  602.106681
           Re-Export  2.655678e+09  8.600322e+08  323.846502
           Re-Import  1.748614e+06  1.257839e+06  719.334856
      2012 Export     1.484154e+10  6.891275e+09  464.323510
           Import     1.594007e+10  9.546980e+09  598.929469

prunit                                                  \
year                   2010        2011        2012        2013        2014   
all   Flow                                                                    
World Export     394.798785  473.898135  464.323510  508.562299  440.711249   
      Import     510.975189  602.106681  598.929469  630.351416  602.151725   
      Re-Export  286.603442  323.846502  326.599208  551.161405  308.400298   
      Re-Import  559.960201  719.334856  749.885775  489.778761  661.093420   

                                                       
year                   2015        2016          2017  
all   Flow                                             
World Export     394.860703  375.820260    428.045101  
      Import     475.202491  424.097682    384.475100  
      Re-Export  319.826580  502.439682  44122.797431  
      Re-Import  357.336028  326.784762    388.655476

In [183]:
pla_wkw_wide = pla_wc[pla_wc.Reporter == 'KOR'].groupby(['Partner', 'Partner_nm', 'Flow', 'year']).agg({'Netweight_kg': np.sum, 'Value_usd': np.sum}).filter(like = 'Netweight_kg', axis = 1).unstack('year')
pla_wkv_wide = pla_wc[pla_wc.Reporter == 'KOR'].groupby(['Partner', 'Partner_nm', 'Flow', 'year']).agg({'Netweight_kg': np.sum, 'Value_usd': np.sum}).filter(like = 'Value_usd', axis = 1).unstack('year')

pla_wkw_wide.head(10)
pla_wkv_wide.head(10)

Netweight_kg                        \
year                                        2010       2011       2012   
Partner Partner_nm           Flow                                        
AGO     ANGOLA               Export          NaN        NaN        NaN   
ARE     UNITED ARAB EMIRATES Export          NaN        NaN        NaN   
                             Import     197785.0        NaN    47600.0   
AUS     AUSTRALIA            Export          NaN        NaN        NaN   
                             Import      61175.0   137765.0     3805.0   
AUT     AUSTRIA              Export          NaN        NaN      345.0   
                             Import          NaN        NaN    13840.0   
BEL     BELGIUM              Export      95860.0        NaN        NaN   
                             Import     111952.0  2107295.0  2878312.0   
BGD     BANGLADESH           Export          NaN   375000.0        NaN   

                                                                      \
year                                      2013       2014       2015   
Partner Partner_nm           Flow                                      
AGO     ANGOLA               Export        1.0        NaN        NaN   
ARE     UNITED ARAB EMIRATES Export        NaN        NaN        NaN   
                             Import    65256.0   116690.0    28980.0   
AUS     AUSTRALIA            Export        NaN        NaN   148780.0   
                             Import   245350.0   257678.0  1750262.0   
AUT     AUSTRIA              Export        NaN        NaN        NaN   
                             Import        NaN        NaN    38180.0   
BEL     BELGIUM              Export        NaN        NaN        NaN   
                             Import  1867365.0  1193090.0  2989475.0   
BGD     BANGLADESH           Export    48720.0        NaN        NaN   

                                                           
year                                      2016       2017  
Partner Partner_nm           Flow                          
AGO     ANGOLA               Export        NaN        NaN  
ARE     UNITED ARAB EMIRATES Export    38905.0        NaN  
                             Import        NaN    26096.0  
AUS     AUSTRALIA            Export   817494.0   294929.0  
                             Import  7841573.0   501118.0  
AUT     AUSTRIA              Export        NaN        NaN  
                             Import        NaN     1720.0  
BEL     BELGIUM              Export        NaN        NaN  
                             Import  2867965.0  2700336.0  
BGD     BANGLADESH           Export        NaN   560550.0

Value_usd                                \
year                                     2010      2011      2012      2013   
Partner Partner_nm           Flow                                             
AGO     ANGOLA               Export       NaN       NaN       NaN   33988.0   
ARE     UNITED ARAB EMIRATES Export       NaN       NaN       NaN       NaN   
                             Import  400747.0       NaN   13050.0   19163.0   
AUS     AUSTRALIA            Export       NaN       NaN       NaN       NaN   
                             Import   40283.0  111468.0    2816.0   51502.0   
AUT     AUSTRIA              Export       NaN       NaN       3.0       NaN   
                             Import       NaN       NaN    7300.0       NaN   
BEL     BELGIUM              Export   23983.0       NaN       NaN       NaN   
                             Import   97929.0  769355.0  987072.0  933736.0   
BGD     BANGLADESH           Export       NaN  391112.0       NaN    3254.0   

                                                                              
year                                     2014       2015      2016      2017  
Partner Partner_nm           Flow                                             
AGO     ANGOLA               Export       NaN        NaN       NaN       NaN  
ARE     UNITED ARAB EMIRATES Export       NaN        NaN   10636.0       NaN  
                             Import  138360.0    17968.0       NaN   26096.0  
AUS     AUSTRALIA            Export       NaN    53404.0  271128.0   48129.0  
                             Import  164728.0   452844.0  229115.0  126919.0  
AUT     AUSTRIA              Export       NaN        NaN       NaN       NaN  
                             Import       NaN    25199.0       NaN    1981.0  
BEL     BELGIUM              Export       NaN        NaN       NaN       NaN  
                             Import  632762.0  1175281.0  526631.0  714529.0  
BGD     BANGLADESH           Export       NaN        NaN       NaN   43599.0

## Assignment
### Preprocessing (no presentation)
+ 제공된 2015년 세계 무역 데이터를 이용하십시오.
    + CSV 파일을 불러오십시오.
    + 값의 형식이 알맞게 불러들여졌는지 확인하십시오. 만약 알맞지 않다면, 적절한 방식으로 변환하십시오. 오류가 없다면 넘어가도 됩니다.
    + 필드 이름에 빈칸이 포함되어 이용하기 불편하므로, 각자가 원하는 방식으로 필드 이름을 새롭게 지정하십시오.
        + 문제에서 이야기했듯이 필드 이름에는 빈칸이 포함되지 않아야 합니다.
        + 만약 빈칸이 들어간 필드를 이용해서 SAC 관련 함수들을 이용하려면, 필드 이름을 \`\` 사이에 넣어야 합니다. (주의: 작은 따옴표 아님! 1 왼쪽의 물결무늬 표시 (~)를 Shift 없이 타이핑할 때 나오는 기호입니다) 
        + 필드 이름은 가급적 숫자로 시작하지 않도록 지정하십시오. 숫자로 시작한 필드도 \`\` 사이에 넣어 처리해야 합니다.
    + `unique` 함수와 파이프 연산자를 이용해서 한국 (ISO code: KOR)의 수입 품목이 몇 가지인지 확인하십시오.
    + 다른 국가의 현황도 살펴보십시오.

### Presentation
+ 본인이 원하는 품목을 한 가지 지정한 후, 위에서 보인 폐플라스틱 예시와 동일한 요령으로 분석하여 5분 이내로 발표하십시오. 강조하여야 할 점은 아래와 같습니다.
    + 지정품목의 전세계 교역량: 중량과 교역액을 분리하여
    + Top X 교역대상국 (수출, 수입 총합): 중량과 교역액을 분리하여
    + Top X 교역대상국 (수출, 수입 분리): 중량과 교역액을 분리하여